# DSCI 100 Project Final Report – Group Component

### 1. Introducton

#### The addressed broad question: "We would like to know which "kinds" of players are most likely to contribute a large amount of data so that we can target those players in our recruiting efforts"

#### Specifc Question: Can age and experience predict the total hours played so we can target similar players for large-data recruitment?
Using provided datasets, We'll investigate this question for Frank Wood's CS research group's game, PlaiCraft. The main dataset used will be players.csv.


 - Collection period: May 1-September 1, 2024
 - Session tracking duration: once game was opened to closed
 - Two NA values (players.csv)
     - I kept them as other variables in those rows may be valuable for visualizations and summaries

<mark>Add more about background info, how data was collected, etc.<mark>

In [9]:
# Now, let's load in some R packages!
library(tidyverse)
library(ggplot2)
library(RColorBrewer)

### 2. Methods & Results

In [10]:
#load the dataset we'll be working with
players <- read_csv("data/players.csv")

head(players)

#let's make the regression model! (steps outlined below)

#1. Mutate "experience"
# 2. Inspect and clean data (eg.handle NA values)
# 3. Split dataset
#     - 75% training, 25% testing
# 4. Tune training set
#     - Scale predictors
# 5. Cross-validate training set
#     - 5 folds
#     - Tested neighbors differ by 4; neighbors = 1,5,...196
# 6. Find optimal *k*, refit model, assess on testing data for RMSPE



Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


experience,subscribe,hashedEmail,played_hours,name,gender,Age
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9
Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17
Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17
Amateur,TRUE,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4fa7a5a659ff443a0eb5,0.7,Flora,Female,21
Regular,TRUE,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb0af4d48fcce2420f3e,0.1,Kylie,Male,21
Amateur,TRUE,f58aad5996a435f16b0284a3b267f973f9af99e7a89bee0430055a44fa92f977,0.0,Adrian,Female,17


#### Description of Above Methods 



#### Further Vizualizations

In [12]:
#Make viualizations based on performed regression

### 3. Discussion

- <mark>Summarize findings
- <mark>Discuss if this what is what was expected
- <mark>What impact do the findings have
- <mark>Are there any future questions of interest

### GitHub Repository

https://github.com/tchan0717/dsci-100-2025w1-group-36.git

### References

Wickham, Hadley, and Garrett Grolemund. 2016. R for Data Science: Import, Tidy, Transform, Visualize, and Model Data. O’Reilly. https://r4ds.had.co.nz/.
\
\
The Pacific Laboratory of Artificial Intelligence. FAQ. Plaicraft. https://plaicraft.ai/faq. 